In [1]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

nlp = pipeline("ner", model=model, tokenizer=tokenizer)
example = "My name is Wolfgang and I live in Berlin"

ner_results = nlp(example)
print(ner_results)

/Users/lukethomas/Documents/searchyenv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Downloading: 100%|██████████| 59.0/59.0 [00:00<00:00, 21.4kB/s]
Downloading: 100%|██████████| 829/829 [00:00<00:00, 609kB/s]
Downloading: 100%|██████████| 213k/213k [00:00<00:00, 852kB/s] 
Downloading: 100%|██████████| 2.00/2.00 [00:00<00:00, 1.65kB/s]
Downloading: 100%|██████████| 112/112 [00:00<00:00, 97.9kB/s]
Downloading: 100%|██████████| 433M/433M [00:06<00:00, 62.7MB/s] 


[{'entity': 'B-PER', 'score': 0.9990139, 'index': 4, 'word': 'Wolfgang', 'start': 11, 'end': 19}, {'entity': 'B-LOC', 'score': 0.999645, 'index': 9, 'word': 'Berlin', 'start': 34, 'end': 40}]


In [1]:
import gunicorn

In [2]:
import sentence_transformers

/Users/lukethomas/Documents/searchy/venv_searchy/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from sentence_transformers import CrossEncoder

In [4]:
from sentence_transformers import CrossEncoder, util

In [5]:
from sentence_transformers import SentenceTransformer, CrossEncoder, util
import torch
import pickle
import pandas as pd
import numpy as np

bi_encoder = SentenceTransformer("multi-qa-MiniLM-L6-cos-v1")
cross_encoder = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")

In [7]:
from searchy import bible_class
#Our sentences we like to encode
with open('bsb2.txt', 'r') as file:
    bible_v = file.readlines()
    print(len(bible_v), 'lines in bsb2')
try:
    embeddings = np.load('bsb_embeddings.npy')
except:
    print('something went wrong')
bible = bible_class(bible_v)
d = embeddings.shape[1]
print('embeddings shape', d)

31102 lines in bsb2
embeddings shape 384


In [38]:
query = 'What is an instance where God shows love?'
question_embedding = bi_encoder.encode(query, convert_to_tensor=True)
hits = util.semantic_search(question_embedding, embeddings, top_k =50)[0]

In [39]:
hits[0]

{'corpus_id': 30615, 'score': 0.44576793909072876}

In [40]:
cross_inp = [[query, bible[hit['corpus_id']]] for hit in hits]
cross_scores = cross_encoder.predict(cross_inp)

# Sort results by the cross-encoder scores
for idx in range(len(cross_scores)):
    hits[idx]['cross-score'] = cross_scores[idx]

hits = sorted(hits, key=lambda x: x['cross-score'], reverse=True)

In [44]:
for i in range(5):
    score = hits[i]['score']
    ref = bible.id2ref[hits[i]['corpus_id']]
    verse = bible[hits[i]['corpus_id']]
    print(f"{score:<.2}|{ref:<15} {verse}")


0.43|1 John 4:16     And we have come to know and believe the love that God has for us. God is love; whoever abides in love abides in God, and God in him.
0.39|1 John 4:8      Whoever does not love does not know God, because God is love.
0.39|Hosea 3:1       Then the LORD said to me, “Go show love to your wife again, though she is loved by another and is an adulteress. Love her as the LORD loves the Israelites, though they turn to other gods and offer raisin cakes to idols.”
0.45|1 John 4:12     No one has ever seen God; but if we love one another, God remains in us, and His love is perfected in us.
0.38|1 John 4:7      Beloved, let us love one another, because love comes from God. Everyone who loves has been born of God and knows God.


In [24]:
for h in hits:
#     print(h['corpus_id'])
    print(bible[h['corpus_id']])

Moses did everything just as the LORD had commanded him.
So the LORD said to Moses,
Thus the Israelites did everything just as the LORD had commanded Moses.
Bezalel son of Uri, the son of Hur, of the tribe of Judah, made everything that the LORD had commanded Moses.
So Moses, Aaron, and the whole congregation of Israel did with the Levites everything that the LORD had commanded Moses they should do.
So Moses did as the LORD had commanded him.
Then the LORD said to Moses,
Then the LORD said to Moses, “Stretch out your hand over the sea, so that the waters may flow back over the Egyptians and their chariots and horsemen.”
So Moses went down to the people and spoke to them.
And all the people answered together, “We will do everything that the LORD has spoken.” So Moses brought their words back to the LORD.
Then the LORD said to Moses,
And when Moses heard this explanation, he was satisfied.
And the LORD said to Moses,
“Moses My servant is dead. Now therefore arise, you and all these peopl

In [ ]:
from sentence_transformers import SentenceTransformer, CrossEncoder, util
import torch
import pickle
import pandas as pd

bi_encoder = SentenceTransformer("multi-qa-MiniLM-L6-cos-v1")
cross_encoder = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")
embeddings=pd.read_pickle("corpus_embeddings_cpu.pkl")
corpus=pd.read_pickle("corpus.pkl")

def search(query,top_k=100):
    print("Top 5 Answer by the NSE:")
    print()
    ans=[]
    ##### Sematic Search #####
    # Encode the query using the bi-encoder and find potentially relevant passages
    question_embedding = bi_encoder.encode(query, convert_to_tensor=True)
    hits = util.semantic_search(question_embedding, corpus_embeddings, top_k=top_k)
    hits = hits[0]  # Get the hits for the first query

    ##### Re-Ranking #####
    # Now, score all retrieved passages with the cross_encoder
    cross_inp = [[query, corpus[hit['corpus_id']]] for hit in hits]
    cross_scores = cross_encoder.predict(cross_inp)

    # Sort results by the cross-encoder scores
    for idx in range(len(cross_scores)):
        hits[idx]['cross-score'] = cross_scores[idx]

    hits = sorted(hits, key=lambda x: x['cross-score'], reverse=True)
    
    for idx, hit in enumerate(hits[0:5]):
        ans.append(corpus[hit['corpus_id']])
    return ans[0],ans[1],ans[2],ans[3],ans[4]

In [ ]:
corpus_embeddings=pd.read_pickle("corpus_embeddings_cpu.pkl")
corpus=pd.read_pickle("corpus.pkl")

def search(query,top_k=100):
    print("Top 5 Answer by the NSE:")
    print()
    ans=[]
    ##### Sematic Search #####
    # Encode the query using the bi-encoder and find potentially relevant passages
    question_embedding = bi_encoder.encode(query, convert_to_tensor=True)
    hits = util.semantic_search(question_embedding, corpus_embeddings, top_k=top_k)
    hits = hits[0]  # Get the hits for the first query

    ##### Re-Ranking #####
    # Now, score all retrieved passages with the cross_encoder
    cross_inp = [[query, corpus[hit['corpus_id']]] for hit in hits]
    cross_scores = cross_encoder.predict(cross_inp)

    # Sort results by the cross-encoder scores
    for idx in range(len(cross_scores)):
        hits[idx]['cross-score'] = cross_scores[idx]

    hits = sorted(hits, key=lambda x: x['cross-score'], reverse=True)
    
    for idx, hit in enumerate(hits[0:5]):
        ans.append(corpus[hit['corpus_id']])
    return ans[0],ans[1],ans[2],ans[3],ans[4]